<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [239]:
import pandas as pd
# Можно было бы, для подключения к БД, воспользоваться библиотекой psycopg2, но
# в таком случае при каждом вызове метода pandas.read_sql_query выходит предупреждение: 
# UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or
# database string URI or sqlite3 DBAPI2 connection. 
# Other DBAPI2 objects are not tested. 
# Please consider using SQLAlchemy.
# Поэтому, в соответствии с данным предупреждением, мной используется библиотека SQLAlchemy 
from sqlalchemy import create_engine, text

In [240]:
# Для подключения берем данные их текстового файла (который не выкладывается на github) и кладем их в словарь:
params = {k.strip():v.strip() for k, v in (l.split('=') for l in open('config.txt'))}

In [241]:
# Формируем строку подключения с заданными параметрами
url=f'''postgresql+psycopg2://{params['USER']}:{params['PASSWORD']}@{params['HOST']}:{params['PORT']}/{params['DBNAME']}'''
# Подключаемся к БД
engine = create_engine(url)
connection = engine.connect()

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [242]:
# текст запроса
query_3_1 = f'''
    select count(*)
    from vacancies
'''

In [243]:
# результат запроса
df = pd.read_sql_query(text(query_3_1), connection)
print(df)

   count
0  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [244]:
# текст запроса
query_3_2 = f'''
    select count(*)
    from employers
'''

In [245]:
# результат запроса
df = pd.read_sql_query(text(query_3_2), connection)
print(df)

   count
0  23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [246]:
# текст запроса
query_3_3 = f'''
    select count(*)
    from areas
'''

In [247]:
# результат запроса
df = pd.read_sql_query(text(query_3_3), connection)
print(df)

   count
0   1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [248]:
# текст запроса
query_3_4 = f'''
    select count(*)
    from industries
'''

In [249]:
# результат запроса
df = pd.read_sql_query(text(query_3_4), connection)
print(df)

   count
0    294


***

*На основании проведенного предварительного анализа данных, можно сделать вывод, что мы имеем объем данных, достаточный для создания модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist.*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [250]:
# текст запроса
query_4_1 = f'''
    select 
        a.name area,
        count(v.id) cnt
    from public.areas a
    join public.vacancies v on v.area_id = a.id
    group by a.name
    order by count(v.id) desc
    limit 5
    '''

In [251]:
# результат запроса
df = pd.read_sql_query(text(query_4_1), connection)
print(df)

              area   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112
3      Новосибирск  2006
4           Алматы  1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [252]:
# текст запроса
query_4_2 = f'''
    select 
        count(v.id)
    from public.vacancies v
    where salary_from is not null or salary_to is not null
    '''

In [253]:
# результат запроса
df = pd.read_sql_query(text(query_4_2), connection)
print(df)

   count
0  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [254]:
# текст запроса
query_4_3 = f'''
    select 
        round(avg(v.salary_from)) mean_salary_from,
        round(avg(v.salary_to)) mean_salary_to
    from public.vacancies v
    '''

In [255]:
# результат запроса
df = pd.read_sql_query(text(query_4_3), connection)
print(df)

   mean_salary_from  mean_salary_to
0           71065.0        110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [256]:
# текст запроса
query_4_4 = f'''
    select 
        v.schedule,
        v.employment,
        count(v.id) cnt
    from public.vacancies v
    group by v.schedule, v.employment
    order by count(v.id) desc 
    '''

In [257]:
# результат запроса
df = pd.read_sql_query(text(query_4_4), connection)
print(df)

            schedule           employment    cnt
0        Полный день     Полная занятость  35367
1   Удаленная работа     Полная занятость   7802
2      Гибкий график     Полная занятость   1593
3   Удаленная работа  Частичная занятость   1312
4     Сменный график     Полная занятость    940
5        Полный день           Стажировка    569
6     Вахтовый метод     Полная занятость    367
7        Полный день  Частичная занятость    347
8      Гибкий график  Частичная занятость    312
9        Полный день     Проектная работа    141
10  Удаленная работа     Проектная работа    133
11     Гибкий график           Стажировка    116
12    Сменный график  Частичная занятость    101
13  Удаленная работа           Стажировка     64
14     Гибкий график     Проектная работа     18
15    Сменный график           Стажировка     12
16    Вахтовый метод     Проектная работа      2
17    Сменный график     Проектная работа      1


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [258]:
# текст запроса
query_4_5 = f'''
    select 
        v.experience,
        count(v.id) cnt
    from public.vacancies v
    group by v.experience
    order by count(v.id) 
    '''

In [259]:
# результат запроса
df = pd.read_sql_query(text(query_4_5), connection)
print(df)

           experience    cnt
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


***

*Детальный анализ вакансий показывает, что в больших городах, таких как Москва, Санкт-Петербург, Минск, Новосибирск, Алматы самый большой спрос на рынке труда. И это логично, чем больше город, тем больше работодателей, и соответсвенно вакансий. В 49% вакансий указана зарплата, средние значения составляют от 71065 до 110537 руб. Набирает актальность удаленная работа, при полной занятости, данный вид графика работы занимает вторую позицию после полного рабочего дня. Наиболее востребованы специалисты с опытом работы от 1 до 3 лет.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [260]:
# текст запроса
query_5_1 = f'''
with cte as 
    (select 
        e.name employee_name, 
        row_number() over (order by count(e.name) desc) as rn 
    from vacancies v 
    join employers e on v.employer_id = e.id group by e.name)
select c.employee_name from cte c where rn in (1, 5)
    '''

In [261]:
# результат запроса
df = pd.read_sql_query(text(query_5_1), connection)
print(df)

   employee_name
0         Яндекс
1  Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [262]:
# текст запроса
query_5_2 = f'''
    select 
        a1.name area,
        sub.cnt_vacancies,
        count(e.id) cnt_employers
    from public.areas a1
    left join public.employers e on e.area = a1.id
    join (select 
                    a2.name area,  
                    count(v.id) cnt_vacancies
                from public.areas a2
                left join public.vacancies v on v.area_id = a2.id
                group by a2.name
                ) sub on sub.area = a1.name
    group by a1.name, sub.cnt_vacancies
    order by sub.cnt_vacancies, count(e.id) desc
    limit 5
    '''

In [263]:
# результат запроса
df = pd.read_sql_query(text(query_5_2), connection)
print(df)

                 area  cnt_vacancies  cnt_employers
0              Россия              0            410
1           Казахстан              0            207
2  Московская область              0             75
3  Краснодарский край              0             19
4  Ростовская область              0             18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [264]:
# текст запроса
query_5_3 = f'''
    select 
        e.name employer,
        count(DISTINCT v.area_id) cnt_areas 
    from public.employers e
    join public.vacancies v on v.employer_id = e.id
    group by e.name
    order by count(v.area_id) desc
    limit 1
    '''

In [265]:
# результат запроса
df = pd.read_sql_query(text(query_5_3), connection)
print(df)

  employer  cnt_areas
0   Яндекс        181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [266]:
# текст запроса
query_5_4 = f'''
    select 
        count(e.id)
    from public.employers e
    left join public.employers_industries i on i.employer_id = e.id
    where i.employer_id is null
    '''

In [267]:
# результат запроса
df = pd.read_sql_query(text(query_5_4), connection)
print(df)

   count
0   8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [268]:
# текст запроса
query_5_5 = f'''
    select 
        e.name employer
    from public.employers e
    left join public.employers_industries i on i.employer_id = e.id
    group by e.name
    having count(i.employer_id) = 4
    order by e.name
    offset 2
    limit 1
        '''

In [269]:
# результат запроса
df = pd.read_sql_query(text(query_5_5), connection)
print(df)

  employer
0     2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [270]:
# текст запроса
query_5_6 = f'''
    select 
        count(e.id) cnt
    from public.employers e
    join public.employers_industries e_i on e_i.employer_id = e.id
    join public.industries i on i.id = e_i.industry_id
    where i.name = 'Разработка программного обеспечения'
    '''

In [271]:
# результат запроса
df = pd.read_sql_query(text(query_5_6), connection)
print(df)

    cnt
0  3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [272]:
# код для получения списка городов-милионников
# Импортируем библиотеку requests
import requests
# Импортируем библиотеку BeautifulSoup
from bs4 import BeautifulSoup
lst_city_million = []
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
# Выполняем GET-запрос, содержимое ответа присваивается переменной response
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) 
# Создаём объект BeautifulSoup, указывая html-парсер
page = BeautifulSoup(response.text, 'html.parser') 
# Получаем контейнер table c классом standard sortable
table = page.find('table', class_='standard sortable')
# Получаем строки таблицы 
lst_tr = table.find_all('tr')
# Циклом идем по кажой строке таблицы
for tr_i in range(len(lst_tr)):
    # Получаем список ячеек текущей строки
    lst_td=lst_tr[tr_i].find_all('td')
    # Циклом идем по каждой ячейке в текущей строке
    for td_i in range(len(lst_td)):
        a = lst_td[td_i].find('a')
        # Если в ячейке есть ссылка, то это значит, что ячейка содержит название города
        if a is not None:
            # Добавляем название города в спискок 
            lst_city_million.append(a.text)
# Выводим на экран полученный список городов-милионников
print(str(lst_city_million)[1:-1])

'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'


In [295]:
# текст запроса
query_5_7 = f'''
    with cte as (
        select 
            a.name, 
            count(v.id) cnt 
        from public.vacancies v 
        join public.employers e on e.id = v.employer_id and e.name = 'Яндекс' 
        join public.areas a on a.id = v.area_id and a.name in ({str(lst_city_million)[1:-1]}) 
        group by a.name
        --order by count(v.id) desc
        ) 
    select 'Total' as name, sum(cnt) from cte c 
    union all 
    select c.name, c.cnt from cte c
    order by 2 desc
    '''

In [296]:
# результат запроса
df = pd.read_sql_query(text(query_5_7), connection)
print(df)

               name    sum
0             Total  485.0
1            Москва   54.0
2   Санкт-Петербург   42.0
3      Екатеринбург   39.0
4   Нижний Новгород   36.0
5       Новосибирск   35.0
6           Воронеж   32.0
7         Краснодар   30.0
8            Самара   26.0
9               Уфа   26.0
10            Пермь   25.0
11           Казань   25.0
12   Ростов-на-Дону   25.0
13        Волгоград   24.0
14       Красноярск   23.0
15        Челябинск   22.0
16             Омск   21.0


***

*На первом месте по количеству вакансий находится компания Яндекс, а также данный работодатель находится на первом месте по количеству регионов, в которых он регистрирует свои вакансии, всего 1933 вакансий, из них 485 в 17 городах милиолнниках. От сюда можно сделать вывод, что Яндекс нанимает специалистов приемущественно на удаленную работу.*

*В регионе Россия зарегистрированы 410 компаний, и ноль вакансий. Это говорит о том, что каждый их этих работодателей географичеки расположен во множестве субъектах РФ, при этом свои ванансии компании регистрируют в отдельных городах.  Например компания ИК СИБИНТЕК, имеющая структуру из множетсова георафических филиалов, зарегистрировавшая вакансии в 46 разных городах. Это делается для того чтобы, не регистрировать все отдельные географические подразделения.*

*У 8419 работодателей не указана сфера деятельности, и это их упущение, т.к. при поиске вакансий по конкретным сферам деятельности данные работодатели не попадут в результат поиска. Скорее всего hr-специалисты не учли этот момент при регистрации своей компании на hh.ru. Остальные 15082 работодателя более ответсвенно отнеслись при регистрации на hh.ru и указали сферы своей деятельности, в примеру у 2ГИС их четыре: Производство мультимедиа, контента, редакторская деятельность, Консалтинговые услуги, Колл-центры, диспетчерские службы, Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее).* 

*3553 компании занимаются разработкой программного обеспечения, данная сфера деятельности имеет самую большую долю среди остальных. По всему видимому, разработчики ПО являются самыми востребованными специалистами на рынке труда.*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [275]:
# текст запроса
query_6_1= f'''
    select 
        count(*)
    from public.vacancies v
    where lower(v.name) like ('%data%') or lower(v.name) like ('%данн%') 
    '''

In [276]:
# результат запроса
df = pd.read_sql_query(text(query_6_1), connection)
print(df)

   count
0   1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [277]:
# текст запроса
query_6_2= f'''
    select 
        count(*)
    from public.vacancies v
    where 
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%') and 
        (lower(v.name) like '%junior%' or
        v.experience = 'Нет опыта' or
        v.employment = 'Стажировка')
    '''

In [278]:
# результат запроса
df = pd.read_sql_query(text(query_6_2), connection)
print(df)

   count
0     51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [279]:
# текст запроса
query_6_3= f'''
    select 
        count(*)
    from public.vacancies v
    where 
        (lower(v.key_skills) like '%sql%' or lower(v.key_skills) like '%postgres%') and
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')
    '''

In [280]:
# результат запроса
df = pd.read_sql_query(text(query_6_3), connection)
print(df)

   count
0    201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [281]:
# текст запроса
query_6_4= f'''
    select 
        count(*)
    from public.vacancies v
    where 
        lower(v.key_skills) like '%python%' and
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')
    '''

In [282]:
# результат запроса
df = pd.read_sql_query(text(query_6_4), connection)
print(df)

   count
0    351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [283]:
# текст запроса
query_6_5= f'''
    select 
        round(avg(length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1), 2)
    from public.vacancies v
    where 
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data  science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')
    '''

In [284]:
# результат запроса
df = pd.read_sql_query(text(query_6_5), connection)
print(df)

   round
0   6.43


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [285]:
# текст запроса
query_6_6= f'''
    select 
        v.experience,
        round(avg(case when salary_from is null then salary_to
             when salary_to is null then salary_from
            else (salary_from + salary_to) / 2
        end)) avg_salary
    from public.vacancies v
    where 
        (v.salary_from is not null or v.salary_to is not null) and
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')
    group by v.experience
    '''

In [286]:
# результат запроса
df = pd.read_sql_query(text(query_6_6), connection)
print(df)

           experience  avg_salary
0           Нет опыта     74643.0
1  От 1 года до 3 лет    139675.0
2       От 3 до 6 лет    243115.0


***

*Предметный анализ показал, что 1771 вакансия имеет отношение к данным, это 3,5% от общего количества вакансий. Но к ним относятся не только DS, но и другие специальности, например аналитики данных, дата-инженеры, администраторы баз данных и прочие. Поэтому данную цифру мы не будем брать во внимание, т.к. нас итерисуюет только DS.*

*Найдена 51 вакансия для начинающих DS, говорит о том, у джунов есть реальные шансы устроиться на работу, но очевидно, что конкуренция большая, т.к. слишком много выпустников онлайн школ по специальности DS. Найдена 201 вакансия с ключевым навыком SQL или postgres, и 351 с требованием знания Python. Это меньше чем вакансий связанных с DS, и это странно, т.к. оба этих интрумента входят в состав основных, скорее всего это говорит о том, что многие работодатели регистрируя вакансии DS подразумевают, что знание SQL и Python, это само самбой разумеющееся. К примеру вакансия Team Lead Data Science, с ключевыми навыками: Data Science Управление командой, или вакансия Senior Data Scientist, с ключевыми навыками: Знание стека DS Знание теории машинного обучения, теории вероятностей, основ математического анализа Основы алгоритмов и структур данных. Судя по назнанию данные вакансии относятся к опытным специалистам. И знание SQL и Python включается в состав стека DS, или даже не обсуждается.*

*В среднем в вакансиях для DS указано 6,43 ключевых навыка. Это говорит, о том что специалист DS должен знать, не только SQL и Python, но и моножество не менее важных навыков, таких как Machine learning, Git, математическая статистика и прочее, для того чтобы делать другие выводы, необходимо анализировать глубже.*
*Средняя зарплата специалиста DS с опытом работы от 3 до 6 месяцев составляет 243115 рублей. Это очень хорошая зарплата учитывая, что средняя зарплата в России по итогам января — марта 2023 года составила 53096 рублей в месяц* 

# Общий вывод по проекту

1. Сделаем запрос показывающий топ-10 сфер деятельности, в которых зарегистрированы вакансии DS.

In [287]:
# текст запроса
query_total_1= f'''
    select
        i.name,
        count(v.id)
    from public.vacancies v
    join public.employers e on e.id = v.employer_id
    join public.employers_industries ei on ei.employer_id = e.id
    join public.industries i on i.id = ei.industry_id
    where
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')    
    group by i.name
    order by count(v.id) desc
    limit 10
    '''

In [288]:
# результат запроса
df = pd.read_sql_query(text(query_total_1), connection)
print(df)

                                                name  count
0                Разработка программного обеспечения    148
1  Системная интеграция,  автоматизации технологи...    139
2                                               Банк     92
3  Интернет-компания (поисковики, платежные систе...     79
4                                    Мобильная связь     23
5  Маркетинговые, рекламные, BTL, дизайнерские, E...     22
6  Управляющая, инвестиционная компания (управлен...     21
7                              Консалтинговые услуги     16
8                                Фиксированная связь     13
9  Научно-исследовательская, научная, академическ...      9


2. Сделаем запрос показывающий топ-10 городов по количеству вакансий DS.

In [289]:
# текст запроса
query_total_2= f'''
    select
        a.name,
        count(v.id)
    from public.vacancies v
    join public.areas a on a.id = v.area_id
    where
        (lower(v.name) like '%data scientist%' or
        lower(v.name) like '%data science%' or
        lower(v.name) like '%исследователь данных%' or
        (v.name like '%ML%' and v.name not like '%HTML%') or
        lower(v.name) like '%machine learning%' or
        lower(v.name) like '%машинн%обучен%')
    group by a.name
    order by count(v.id) desc 
    limit 10
    '''

In [290]:
# результат запроса
df = pd.read_sql_query(text(query_total_2), connection)
print(df)

              name  count
0           Москва    217
1  Санкт-Петербург     64
2      Новосибирск     23
3  Нижний Новгород     20
4           Казань     16
5           Алматы     16
6            Минск      9
7            Томск      8
8     Екатеринбург      7
9          Армения      6


3. Сделаем запрос показывающий топ-10 ключевых навыков, которые указаны вакансиях DS.

In [291]:
# текст запроса
query_total_3= f'''
    SELECT ks key_skills, count(v.id) FROM public.vacancies v, regexp_split_to_table(lower(v.key_skills), chr(9)) ks
        where 
            (lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data  science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%')
    group by ks
    order by count(v.id) desc
    limit 10
    '''

In [292]:
# результат запроса
df = pd.read_sql_query(text(query_total_3), connection)
print(df)

                  key_skills  count
0                     python    323
1                        sql    175
2           machine learning    133
3                        git     64
4                    pytorch     61
5  математическая статистика     58
6                     pandas     52
7              data analysis     50
8               data science     50
9                         ml     45


4. Сделаем запрос показывающий распределение графиков работы по количеству вакансий DS.

In [293]:
# текст запроса
query_total_5= f'''
    SELECT schedule, count(v.id) 
    FROM public.vacancies v
        where 
            (lower(v.name) like '%data scientist%' or
            lower(v.name) like '%data  science%' or
            lower(v.name) like '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            lower(v.name) like '%machine learning%' or
            lower(v.name) like '%машинн%обучен%')
    group by schedule
    order by count(v.id) desc
    '''

In [294]:
# результат запроса
df = pd.read_sql_query(text(query_total_5), connection)
print(df)

           schedule  count
0       Полный день    293
1  Удаленная работа    111
2     Гибкий график     42


*Самой распространенной сферой деятельности, в которой востребованы специалисты DS является Разработка программного обеспечения. И это логично, специалисты DS занимаются разработкой скриптов Python и SQL для выгрузки и подготовки данных к анализу, разрабатывают модели машинного обучения, все это относится к разработке. Вероятно компании разработчики ПО предлагают готовые типовые решения в области DS для интернет магазинов, операторов связи, банков и прочих коммерческих структур, обладающих BigData, выполняют индивидуальные разработки под заказ.*

*В Москве наиболее всего востребованы специалисты DS, во первых это самый большой город, и здесь больше всего работодателей, во вторых, услуги DS нужды прежде всего для крупных структур собирающих большие объемы данных, а такие структуры наиболее всего сосредаточены в крупных городах.*

*Как уже говорилось выше, область DS не ограничивается Python и SQL, это самые основные навыки, за ними идут, ML, Git, Pythorch (фреймворк Python для ML), математическая статистика, анализ данных, Pandas.*

*Найдено 111 вакансий с возможностью удаленной работы. В свете последних событий связанных с COVID, удаленная работа набирает популярность. В данном режиме работы множество плюсов как для работодателя так и для работника.*